# Imports and Functions

In [ ]:
"""
This module handles AI-powered mind map generation using the Gemini API.
It includes utilities for retrying API calls, environment variable management,
and Jupyter notebook display enhancements.
"""

# Standard library imports
import os
import time
import datetime
import subprocess
import threading
import itertools
import re
import logging

# Third-party imports
from tenacity import retry, stop_after_attempt, wait_exponential
from dotenv import load_dotenv
import google.generativeai as genai

# Jupyter-specific imports
from IPython.display import Image, display, clear_output
from ipywidgets import widgets, IntProgress, HBox, Label, Button, Layout

# Load environment variables from 'keys.env' file
# This file should contain the GOOGLE_API_KEY
load_dotenv("keys.env")
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Configure the Gemini model using the API key
# Ensure the API key is valid and has necessary permissions
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize the Gemini model
# Using 'gemini-1.5-flash' for faster processing
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

# Define the prompt_text for mind map generation
# This multi-line string contains instructions for the AI model
prompt_text = '''
You are a skilled document analyst. Your task is to create a mind map from a given PDF document in .puml format suitable for use in PlantUML. Adhere strictly to the following guidelines:

Step 1: Read the PDF document carefully, maintaining the original language throughout your analysis.

Step 2: Structure the mind map with explicit levels of topics and subtopics:
  - Use '*', '**', '***', '****', '*****', etc., for each level, adding one asterisk per level.
  - Levels 1 and 2: Limit to 1-10 words per topic.
  - Levels 3 and 4: Allow 1-20 words per topic, forming complete phrases.
  - Ensure the mind map has at least 4 levels, but feel free to add more levels if needed to fully explore the concepts.

Step 3: Ensure the resulting .puml file has no more than 90 lines, ideally filling 60-80 lines, without using '-' at the beginning of each topic.

Step 4: Exclude any sections that do not directly explain concepts, such as final considerations, complementary materials, or bibliographic references.

Step 5: Mandatory use of multi-line structure for complex topics:
  - For any topic that requires more than one line of explanation, use the following format:
    ***: First line of the explanation
    Second line continuing the explanation
    Third line if necessary;
  - Always start with asterisks and ':', and end with ';' in multi-line.
  - Use this structure for at least 40% of your topics, especially in levels 3 and beyond.

Step 6: 
  
Example (please pay attention on the sintax!):
  @startmindmap
  * Subject
  ** Key Concept
  ***: Definition of the concept -
  its main characteristics or
  how it's applied in practice;
  ****: Detailed Example -
  Step 1 of the example
  Step 2 of the example
  Conclusion of the example;
  ** History
  *** Period X
  *** Period Y
  ** Applications
  *** Application X
  *** Application Y
  *** Application Z
  @endmindmap

Remember to use the multi-line structure frequently throughout your mind map to enhance clarity and depth of explanation.
'''

def check_files_in_genai_cloud():
    """
    Check if there are any files present in the Gemini AI cloud storage.

    This function uses the Gemini AI API to list files in the cloud storage
    and returns True if at least one file is present, False otherwise.

    Returns:
        bool: True if there are files in the cloud storage, False if it's empty.

    Raises:
        APIError: If there's an error in communicating with the Gemini AI API.

    Example:
        >>> has_files = check_files_in_genai_cloud()
        >>> print(has_files)
        True
    """
    return any(genai.list_files())

def remove_files_from_cloud():
    """
    Remove all files from the Gemini AI cloud storage.

    This function iterates through all files in the Gemini AI cloud storage
    and attempts to delete each one. It provides feedback on the success or
    failure of each deletion operation.

    Returns:
        None

    Raises:
        APIError: If there's an error in communicating with the Gemini AI API.

    Side Effects:
        - Deletes files from the Gemini AI cloud storage.
        - Prints status messages to the console for each file deletion attempt.

    Example:
        >>> remove_files_from_cloud()
        The file "document1.pdf" was removed from the cloud.
        The file "image1.jpg" was removed from the cloud.
        Error removing file "locked_file.txt": Permission denied
    """
    for file in genai.list_files():
        try:
            genai.delete_file(file.name)
            print(f"The file {file.display_name} was removed from the cloud.")
        except Exception as e:
            print(f"Error removing file {file.display_name}: {e}")

def upload_files_to_cloud(inputs_path):
    """
    Upload PDF files from a specified directory to the Gemini AI cloud storage.

    This function scans the given directory for PDF files, sorts them by creation time,
    and uploads them to the Gemini AI cloud storage. It provides feedback on the success
    or failure of each upload operation.

    Args:
        inputs_path (str): The path to the directory containing PDF files to upload.

    Returns:
        list: A list of successfully uploaded file objects from the Gemini AI API.

    Raises:
        APIError: If there's an error in communicating with the Gemini AI API.

    Side Effects:
        - Uploads files to the Gemini AI cloud storage.
        - Prints status messages to the console for each file upload attempt.
        - Introduces a 1-second delay between uploads to avoid overwhelming the API.

    Example:
        >>> uploaded = upload_files_to_cloud("/path/to/pdf/directory")
        Uploaded file 'document1.pdf' as: gemini://abc123
        Uploaded file 'document2.pdf' as: gemini://def456
        Error uploading file large_file.pdf: File size exceeds limit
        >>> print(len(uploaded))
        2
    """
    if not os.path.exists(inputs_path):
        print(f"Directory {inputs_path} does not exist.")
        return []

    files = [os.path.join(inputs_path, pdf) for pdf in os.listdir(inputs_path) if pdf.endswith('.pdf')]
    files.sort(key=os.path.getctime)
    
    uploaded_files = []
    for path in files:
        pdf_name = os.path.basename(path)
        try:
            file = genai.upload_file(path=path, display_name=pdf_name)
            print(f"Uploaded file '{file.display_name}' as: {file.uri}")
            uploaded_files.append(file)
            time.sleep(1)
        except Exception as e:
            print(f"Error uploading file {pdf_name}: {e}")
    
    return uploaded_files

def save_to_file(text, file):
    """
    Save the content of the text variable to a text file in the 'outputs' folder.

    :param text: The text to be saved to the file.
    :param file: The file object received as input.
    """
    try:      
        # Extract the file name from the file object
        filename = os.path.basename(file.display_name[:-4])+".txt"      
        timestamp_txt = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        # Create the full path for the file in the 'outputs' folder
        output_path = os.path.join('outputs', timestamp_txt+filename)
        # Open the file in write mode
        with open(output_path, 'w', encoding='utf-8') as file_txt:
            file_txt.write(text)
        print(f"Content successfully saved to the file '{output_path}'.")
    except IOError as e:
        print(f"Error saving the file: {e}")

def ensure_puml_pattern(text):
    # Regex pattern to capture the PlantUML structure
    pattern_puml_mindmap = r'(@startmindmap[\s\S]*?@endmindmap)'
    match_puml = re.search(pattern_puml_mindmap, text, re.DOTALL)
    
    if not match_puml:
        print("No mind map found in the text.")
        return
    
    match_puml_text = match_puml.group(1)
    
    # This is crucial to avoid syntax issues in the final diagram
    pattern = r'(\*+)( |:)([\s\S]*?)(?=\n\*+|@endmindmap)'
    
    # Use re.DOTALL and re.MULTILINE to ensure all relevant lines are captured
    matches = re.finditer(pattern, match_puml_text, re.DOTALL | re.MULTILINE)
    
    processed_lines = []
    # Iterate over the matches, maintaining the hierarchical structure
    for match in matches:
        asterisks, separator, content = match.groups()
        line = f"{asterisks}{separator} {content.strip()}"
        processed_lines.append(line)
    
    # Critical processing for compatibility with PlantUML
    final_lines = []
    for line in processed_lines:      
        if '\n' not in line:
            # For single lines:
            # 1. Remove ':' after asterisks to avoid syntax errors
            # 2. Replace " : " with " - " to maintain readability
            # 3. Remove remaining ':' that may cause issues in PlantUML
            line = re.sub(r'^(\*+):?\s*', r'\1 ', line)
            line = line.replace(" : ", " - ")
            line = line.rstrip(".;:")
            final_lines.append(line)
        else:
            # For multi-line:
            line = re.sub(r'^(\*+) ', r'\1: ', line)
            line = line.replace(";",",")
            line = line.rstrip(".:")
            line = line+";"
            line = line.replace(",;",";")
            final_lines.append(line)
    
    final_lines.insert(0,"@startmindmap")
    final_lines.append("@endmindmap")

    return final_lines

# Execute the prompt using the Gemini API, retrieve the response and ensure the response will be in puml mindmap format
def run_API(file, prompt):
    """
    Execute a prompt using the Gemini API and return a PUML mindmap.

    This function sends the given prompt to the Gemini API, retrieves the response,
    and ensures that the response is formatted as a PUML mindmap.

    Args:
        file: The file content to be used with the prompt.
        prompt (str): The prompt to send to the Gemini API.

    Returns:
        tuple: A tuple containing:
            - str: A PUML-formatted mindmap string.
            - list: A list of processed lines.

    Raises:
        ValueError: If the response is not in the correct PUML mindmap format.
    """

    try:
        response = model.generate_content([file, prompt])
        text = response.text
        save_to_file(text, file)
        final_lines = ensure_puml_pattern(text)
    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")
        print(f"Error: {str(e)}")

    final_lines = ensure_puml_pattern(text)

    # Join the processed lines
    str_puml = '\n'.join(final_lines)

    return str_puml, final_lines

# Check if the created map has at least one level 4 subtopic
def check_criteria(lines):
    """
    Check if the generated mind map has at least one level 4 subtopic.

    This function examines a list of strings representing a mind map structure
    and verifies if there's at least one line starting with four asterisks,
    which indicates a level 4 subtopic in the PUML mind map format.

    Args:
        lines (list of str): A list of strings representing the lines of a PUML mind map.

    Returns:
        bool: True if at least one level 4 subtopic is found, False otherwise.

    Example:
        >>> mind_map_lines = [
        ...     "@startmindmap",
        ...     "* Root",
        ...     "** Level 2",
        ...     "*** Level 3",
        ...     "**** Level 4",
        ...     "@endmindmap"
        ... ]
        >>> result = check_criteria(mind_map_lines)
        >>> print(result)
        True

        >>> simple_map = [
        ...     "@startmindmap",
        ...     "* Root",
        ...     "** Level 2",
        ...     "*** Level 3",
        ...     "@endmindmap"
        ... ]
        >>> result = check_criteria(simple_map)
        >>> print(result)
        False
    """
    return any(line.startswith("****") for line in lines)

# Retry file processing up to 5 times with exponential backoff, ensuring generated PUML meets criteria
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=4, max=10))
def process_file_with_retry(file, prompt):
    """
    Process a file using the Gemini API with retry logic, ensuring the generated mind map meets specific criteria.

    This function attempts to generate a PUML mind map using the Gemini API and checks if the result
    meets the required criteria (having at least one level 4 subtopic). It uses a retry mechanism
    to handle potential failures or unsatisfactory results.

    Args:
        file: The file object to be processed by the Gemini API.
        prompt (str): The prompt to be used for generating the mind map.

    Returns:
        tuple: A tuple containing two elements:
            - str: The PUML-formatted mind map string.
            - list: The lines of the PUML mind map.

    Raises:
        ValueError: If the generated PUML does not meet the criteria after all retry attempts.
        APIError: If there's a persistent error in communicating with the Gemini API.

    Retry Behavior:
        - Maximum attempts: 5
        - Wait time between retries: Exponential backoff
            - Starting at 4 seconds
            - Multiplying by 1 each attempt
            - Up to a maximum of 10 seconds
    """
    str_puml, lines = run_API(file, prompt)
    if not check_criteria(lines):
        raise ValueError("Generated PUML does not meet criteria")
    return str_puml, lines

def generate_mind_map(str_puml, file_name):
    """
    Generate a mind map image from a PUML string and display it.

    This function takes a PUML-formatted string representing a mind map and a file name,
    saves the PUML content to a file, converts it to a PNG image using PlantUML,
    and displays the resulting image.

    Args:
        str_puml (str): A string containing the PUML-formatted mind map.
        file_name (str): The name of the original file, used to name the output files.

    Returns:
        None

    Side Effects:
        - Creates a directory named 'outputs' if it doesn't exist.
        - Writes a .puml file in the 'outputs' directory.
        - Generates a .png file from the .puml file using PlantUML.
        - Displays the generated PNG image if successful.
        - Prints status messages to the console.

    Raises:
        IOError: If there are issues writing the PUML file or running PlantUML.

    Dependencies:
        - Requires PlantUML jar file located at "plantuml_jar\\plantuml-1.2024.7.jar".
        - Depends on an external `run_plantuml()` function to convert PUML to PNG.

    Example:
        >>> puml_content = "@startmindmap\\n* Root\\n** Branch 1\\n** Branch 2\\n@endmindmap"
        >>> generate_mind_map(puml_content, "example_file.pdf")
        # This will create and display a mind map image based on the PUML content
    """
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = "outputs"
    os.makedirs(output_dir, exist_ok=True)
    
    path_puml = os.path.join(output_dir, f"{timestamp}_mapa_mental_{os.path.splitext(file_name)[0]}.puml")
    path_png = path_puml[:-4] + "png"
    
    with open(path_puml, "w", encoding="utf-8") as f:
        f.write(str_puml)
        print(f"You can see the .puml generated in: {path_puml}")
    
    plantuml_jar_path = "plantuml_jar\\plantuml-1.2024.7.jar"
    if run_plantuml(path_puml, plantuml_jar_path):
        if os.path.exists(path_png):
            # NOTE: Image display functionality temporarily removed.
            # To re-enable, uncomment the following line:
            # display(Image(path_png))
            # This might be useful for debugging or when direct visualization in the notebook is needed.
            print(f"You can see the Image generated in: {path_png}")
        else:
            print("Image not found. Check if PlantUML correctly converted the .puml file to .png.")
    else:
        print("Failed to generate the image.")

def run_plantuml(path_puml, plantuml_jar_path):
    """
    Execute PlantUML to convert a PUML file to an image.

    This function uses subprocess to run the PlantUML jar file, which converts
    the specified PUML file to an image (typically PNG). It captures and reports
    any errors that occur during the conversion process.

    Args:
        path_puml (str): The file path to the PUML file to be converted.
        plantuml_jar_path (str): The file path to the PlantUML jar file.

    Returns:
        bool: True if the image was generated successfully, False otherwise.

    Raises:
        subprocess.CalledProcessError: If the PlantUML process exits with a non-zero status.
        
    Side Effects:
        - Executes a Java subprocess to run PlantUML.
        - Generates an image file in the same directory as the input PUML file.
        - Prints status messages to the console.

    Example:
        >>> puml_file = "path/to/mindmap.puml"
        >>> plantuml_jar = "path/to/plantuml.jar"
        >>> success = run_plantuml(puml_file, plantuml_jar)
        >>> if success:
        ...     print("Mind map image created successfully")
        ... else:
        ...     print("Failed to create mind map image")

    Note:
        This function requires Java to be installed and accessible from the command line.
        It uses UTF-8 encoding for file handling, which should be suitable for most use cases.
    """
    try:
        result = subprocess.run(
            ["java", "-Dfile.encoding=UTF-8", "-jar", plantuml_jar_path, path_puml],
            check=True,
            capture_output=True,
            text=True
        )
        print("Image generated successfully!")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error generating image: {e}")
        print(f"STDOUT: {e.stdout}")
        print(f"STDERR: {e.stderr}")
        return False

def spinning_indicator():
    """
    Generate a continuous sequence of spinning indicator characters.

    This function creates an infinite iterator that cycles through a sequence
    of characters representing a spinning animation: '-', '/', '|', '\'.

    Yields:
        str: A single character representing the next frame of the spinning animation.

    Example:
        >>> spinner = spinning_indicator()
        >>> for _ in range(8):
        ...     print(next(spinner), end='', flush=True)
        ...     time.sleep(0.1)
        -/|\\-/|
    
    Note:
        This function is typically used in conjunction with other functions
        to create a visual spinning indicator for long-running processes.
    """
    spinner = itertools.cycle(['-', '/', '|', '\\'])
    while True:
        yield next(spinner)

def update_spinner_controlled(spinner_label, running_event):
    """
    Update a spinner label with a spinning animation, controlled by an event.

    This function updates a given label with a spinning animation. The animation
    continues until the provided event is cleared. This is typically used to
    provide visual feedback for a long-running process in a GUI or notebook environment.

    Args:
        spinner_label: A label object that has a 'value' attribute which can be updated.
                       This is typically a widget in a GUI or notebook environment.
        running_event (threading.Event): An event object used to control the duration
                                         of the spinning animation.

    Side Effects:
        - Continuously updates the 'value' attribute of spinner_label.
        - Sleeps for short intervals between updates.
        - Clears the spinner_label when the animation stops.

    Example:
        >>> import threading
        >>> from ipywidgets import Label
        >>> spinner_label = Label(value='')
        >>> running_event = threading.Event()
        >>> running_event.set()
        >>> # In a separate thread:
        >>> update_spinner_controlled(spinner_label, running_event)
        >>> # To stop the spinner:
        >>> running_event.clear()

    Note:
        This function is designed to be run in a separate thread to avoid
        blocking the main execution. The spinning animation will continue
        until the running_event is cleared.
    """
    for c in spinning_indicator():
        if not running_event.is_set():
            break
        spinner_label.value = c
        time.sleep(0.1)
    spinner_label.value = ''  # Clears the spinner when the thread finishes

def process_files(files):
    """
    Process a list of files to generate mind maps with visual progress indicators.

    This function iterates through a list of files, processes each one to generate
    a mind map, and provides visual feedback on the progress using a progress bar,
    a spinning indicator, and status labels. It includes error handling and a
    10-second pause between file processing.

    Args:
        files (list): A list of file objects to be processed. Each file object
                      should have a 'display_name' attribute.

    Returns:
        None

    Side Effects:
        - Displays a progress bar, spinner, and status label in the current output area.
        - Processes each file to generate a mind map.
        - Prints status messages and error information to the console.
        - Introduces a 10-second pause between processing each file.

    Raises:
        Exception: Any exception raised during file processing is caught and logged.

    Dependencies:
        - Requires IPython widgets (IntProgress, Label, HBox) for visual feedback.
        - Uses threading for the spinner animation.
        - Depends on external functions: update_spinner_controlled, process_file_with_retry,
          generate_mind_map.
        - Assumes the existence of a global variable 'prompt_text'.

    Example:
        >>> files_to_process = [file1, file2, file3]  # Assuming these are valid file objects
        >>> process_files(files_to_process)
        # This will display a progress bar and process each file, generating mind maps

    Note:
        This function is designed to be run in an interactive environment that supports
        IPython widgets, such as Jupyter Notebook or JupyterLab.
    """
    progress = IntProgress(min=0, max=len(files), description='Files:')
    spinner_label = Label(value='-')
    status_label = Label(value='Processing...')
    
    display(HBox([progress, spinner_label, status_label]))
    
    # Variable to control the execution of the spinner thread
    spinner_running = threading.Event()
    spinner_running.set()  # Initially set to True

    spinner_thread = threading.Thread(target=update_spinner_controlled, args=(spinner_label, spinner_running))
    spinner_thread.daemon = True
    spinner_thread.start()

    try:
        total_files = len(files)
        for i, file in enumerate(files):
            status_label.value = f'Processing {file.display_name}...'
            
            try:
                str_puml, _ = process_file_with_retry(file, prompt_text)
                generate_mind_map(str_puml, file.display_name)
            except Exception as e:
                print(f"Failed to process {file.display_name}: {e}")
            finally:
                progress.value = i + 1
                if i < total_files - 1:  # Se não for o último arquivo
                    print("10-second suspension initiated...")
                    for remaining in range(10, 0, -1):
                        status_label.value = f"10-second suspension initiated... {remaining} seconds remaining"
                        time.sleep(1)
                else:
                    print("All files processed.")
        
        status_label.value = 'Finished!'
    finally:
        spinner_running.clear()  # Signal the spinner thread to stop
        spinner_thread.join(timeout=1)  # Wait for the spinner thread to finish (with timeout)

    print("Script finished! All files in the 'pdf' folder have been scanned to generate mind maps.")

def run_workflow(upload_new_files=True):
    """
    Execute the complete workflow for generating mind maps from files.

    This function orchestrates the entire process of uploading files (if needed),
    listing files, and processing them to generate mind maps. It can either upload
    new files or use existing files in the cloud, depending on the input parameter.

    Args:
        upload_new_files (bool, optional): If True, upload new files from the local
                                           directory. If False, use existing files
                                           in the cloud. Defaults to True.

    Returns:
        None

    Side Effects:
        - Modifies the global variable 'sorted_files'.
        - Uploads files to the cloud or lists existing files.
        - Removes existing files from the cloud if uploading new files.
        - Prints status messages to the console.
        - Initiates the file processing and mind map generation process.

    Raises:
        Various exceptions may be raised by the called functions, particularly
        related to file operations and API interactions.

    Dependencies:
        - Requires global variables: sorted_files, inputs_path
        - Uses functions: check_files_in_genai_cloud, remove_files_from_cloud,
          upload_files_to_cloud, genai.list_files, process_files

    Example:
        >>> run_workflow()  # Upload new files and process them
        >>> run_workflow(upload_new_files=False)  # Use existing files in the cloud

    Note:
        This function assumes the existence of a cloud storage system (likely
        Gemini AI) and local input directory. It's designed to be the main
        entry point for the mind map generation workflow.
    """
    global sorted_files
    
    if upload_new_files:
        if check_files_in_genai_cloud():
            remove_files_from_cloud()
        
        uploaded_files = upload_files_to_cloud(inputs_path)
        sorted_files = sorted(uploaded_files, key=lambda file: file.create_time)
        
        print("Uploaded files:")
        for file in sorted_files:
            print(f"{file.display_name} - {file.create_time}")
    else:
        if not check_files_in_genai_cloud():
            print("No files in the cloud. Please upload new files.")
            return
        
        sorted_files = sorted(list(genai.list_files()), key=lambda file: file.create_time)
        print("Using existing files in the cloud:")
        for file in sorted_files:
            print(f"{file.display_name} - {file.create_time}")
    
    print("Starting map generation...")
    process_files(sorted_files)

def list_puml_files():
    output_dir = "outputs"
    puml_files = [f for f in os.listdir(output_dir) if f.endswith('.puml')]
    
    if not puml_files:
        return widgets.HTML("No .puml file found in the 'outputs' folder.")
    
    dropdown = widgets.Dropdown(
        options=puml_files,
        description='PUML Files:',
        style={'description_width': 'initial'}
    )
    
    generate_button = widgets.Button(
        description="Generate Mind Map",
        button_style='primary',
        tooltip='Click to generate the mind map'
    )
    
    output = widgets.Output()
    
    def on_button_click(b):
        with output:
            output.clear_output()
            if dropdown.value:
                generate_and_display_image(dropdown.value)
            else:
                print("Please select a file before generating the mind map.")
    
    generate_button.on_click(on_button_click)
    
    return widgets.VBox([widgets.HBox([dropdown, generate_button]), output])

def generate_and_display_image(selected_file):
    output_dir = "outputs"
    file_path = os.path.join(output_dir, selected_file)
    plantuml_jar_path = "plantuml_jar\\plantuml-1.2024.7.jar"
    
    command = f'java -Dfile.encoding=UTF-8 -jar "{plantuml_jar_path}" "{file_path}"'
    exit_code = os.system(command)
    
    if exit_code == 0:
        print("Image generated successfully!")
    else:
        print("There was an error generating the image.")
        return
    
    image_path = file_path[:-4] + "png"
    if os.path.exists(image_path):
        display(Image(image_path))
    else:
        print("Image not found. Check if PlantUML correctly converted the .puml file to .png.")

# User Interface and Workflow Setup

In [ ]:
# User interface
def create_combined_interface():
    upload_button = widgets.Button(description="Upload and Process", layout=widgets.Layout(width='150px'))
    reprocess_button = widgets.Button(description="Reprocess Uploaded Files", layout=widgets.Layout(width='200px'))
    
    upload_button.on_click(lambda _: run_workflow(True))
    reprocess_button.on_click(lambda _: run_workflow(False))
    
    workflow_buttons = widgets.HBox([upload_button, reprocess_button])
    puml_selector = list_puml_files()
    
    return widgets.VBox([workflow_buttons, puml_selector])

# Inicialização
inputs_path = "inputs"
combined_interface = create_combined_interface()
display(combined_interface)